# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [10]:
#from tensorflow import keras
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.layers import SimpleRNN
#from tensorflow.keras.datasets import imdb
#from tensorflow.keras import initializers
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers
import tensorflow as tf

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32 # Thus will go through many iterations of gradient descent before an epoch

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features) #Max features is just the 20,000 most common words within our dataset
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

17473536/17464789 [==============================] - 0s 0us/step
25000 train sequences
25000 test sequences


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim` i.e 50 in this case.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe). So in our case will be specific to imdb.
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects. Going to keep at 30 by padding or truncating accordingly.

`word_embedding_dim = 5` so input dimension is a vector of 50 numbers, one vector should be close to another if they are similar in meaning.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- kernel_initializer, is our initial weight values for our matrices (input). Same with recurrent_initializer, weights for state layers.
- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [6]:
# Confirming our input shape
x_train.shape[1:]

(30,)

In [7]:
## Let's build a RNN
# Note here we have changed our activation function to ReLu

rnn_hidden_dim = 5
word_embedding_dim = 50 # Take our integers and come up with an embedding where it will transfer each integer into a vector of dim 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
# Initializing weights for the first layer (kernel), and state layer (recurrent).
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))
# Dense layer provides us with the classification output
model_rnn.add(Dense(1, activation='sigmoid'))

In [8]:
## Note that most of the parameters come from the embedding layer
# Simple rnn layer, initial matrix input to state should have a 50 input, 5 hidden, add on bias term 50x5 +5 for the bias term so 255 then we use 5x5 matrix from one layer to the next so 280.
# Dense layer, 5 input plus the bias term.
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [12]:
rmsprop = tf.keras.optimizers.RMSprop(learning_rate = .0001)

# binary_crossentropy as we are looking at a classification output
model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [13]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 8s 9ms/step - loss: 0.6586 - accuracy: 0.6281 - val_loss: 0.5990 - val_accuracy: 0.6879
Epoch 2/10
782/782 [==============================] - 7s 9ms/step - loss: 0.5520 - accuracy: 0.7230 - val_loss: 0.5477 - val_accuracy: 0.7159
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4944 - accuracy: 0.7634 - val_loss: 0.5098 - val_accuracy: 0.7465
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4553 - accuracy: 0.7870 - val_loss: 0.4878 - val_accuracy: 0.7592
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4267 - accuracy: 0.8042 - val_loss: 0.4748 - val_accuracy: 0.7671
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4059 - accuracy: 0.8161 - val_loss: 0.4629 - val_accuracy: 0.7765
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 0.3903 - accuracy: 0.8251 - val_loss: 0.4541 - val_accuracy: 0.7818
Epoch 

In [14]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 2s 2ms/step - loss: 0.4483 - accuracy: 0.7874
Test score: 0.44827160239219666
Test accuracy: 0.7874400019645691


score is the binary cross entropy loss (log loss)

## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [15]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this to see difference in performance compared to above

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [16]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [17]:
rmsprop = tf.keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [18]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 15s 18ms/step - loss: 0.5934 - accuracy: 0.6782 - val_loss: 0.5073 - val_accuracy: 0.7557
Epoch 2/10
782/782 [==============================] - 14s 18ms/step - loss: 0.4475 - accuracy: 0.7930 - val_loss: 0.4472 - val_accuracy: 0.7877
Epoch 3/10
782/782 [==============================] - 15s 19ms/step - loss: 0.3791 - accuracy: 0.8331 - val_loss: 0.4071 - val_accuracy: 0.8145
Epoch 4/10
782/782 [==============================] - 15s 19ms/step - loss: 0.3391 - accuracy: 0.8550 - val_loss: 0.3989 - val_accuracy: 0.8202
Epoch 5/10
782/782 [==============================] - 13s 16ms/step - loss: 0.3138 - accuracy: 0.8684 - val_loss: 0.3885 - val_accuracy: 0.8248
Epoch 6/10
782/782 [==============================] - 14s 18ms/step - loss: 0.2944 - accuracy: 0.8794 - val_loss: 0.3753 - val_accuracy: 0.8315
Epoch 7/10
782/782 [==============================] - 13s 17ms/step - loss: 0.2787 - accuracy: 0.8858 - val_loss: 0.3855 - val_accuracy:

<b> Result comparison between maxlen between 30 vs 80 </b>
- 30 has a result of 0.787
- 80 has an accuracy of 0.837.

So a longer sentence has a better accuracy rate.

In [23]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [24]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [25]:
rmsprop = tf.keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [26]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 13s 15ms/step - loss: 0.6679 - accuracy: 0.5524 - val_loss: 0.6261 - val_accuracy: 0.7265
Epoch 2/10
782/782 [==============================] - 11s 14ms/step - loss: 0.5976 - accuracy: 0.7384 - val_loss: 0.6059 - val_accuracy: 0.7048
Epoch 3/10
782/782 [==============================] - 12s 15ms/step - loss: 0.5613 - accuracy: 0.7810 - val_loss: 0.5384 - val_accuracy: 0.7648
Epoch 4/10
782/782 [==============================] - 12s 15ms/step - loss: 0.4805 - accuracy: 0.7817 - val_loss: 0.5446 - val_accuracy: 0.7528
Epoch 5/10
782/782 [==============================] - 13s 17ms/step - loss: 0.4461 - accuracy: 0.7992 - val_loss: 0.5300 - val_accuracy: 0.7629
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4253 - accuracy: 0.8084 - val_loss: 0.4799 - val_accuracy: 0.7850
Epoch 7/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4099 - accuracy: 0.8188 - val_loss: 0.4740 - val_accuracy:

<b> Result comparison between max number of features </b>\
Comparing top 20,000 features vs 5,000
- 20,000 has a result of 0.787
- 5,000 has an accuracy of 0.803.

So in our case the top 5,000 words has a better result.
Note rember if our accuracy is still improbing at the end of our epochs we should increase the number.

In [28]:
# Out of curiosity, run for 10 more epochs, this will run for 10 more
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 11s 15ms/step - loss: 0.3662 - accuracy: 0.8420 - val_loss: 0.4233 - val_accuracy: 0.8091
Epoch 2/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3584 - accuracy: 0.8471 - val_loss: 0.4177 - val_accuracy: 0.8117
Epoch 3/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3518 - accuracy: 0.8498 - val_loss: 0.4285 - val_accuracy: 0.8082
Epoch 4/10
782/782 [==============================] - 11s 15ms/step - loss: 0.3454 - accuracy: 0.8530 - val_loss: 0.4109 - val_accuracy: 0.8142
Epoch 5/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3393 - accuracy: 0.8555 - val_loss: 0.4046 - val_accuracy: 0.8167
Epoch 6/10
782/782 [==============================] - 12s 15ms/step - loss: 0.3330 - accuracy: 0.8577 - val_loss: 0.4024 - val_accuracy: 0.8170
Epoch 7/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3274 - accuracy: 0.8607 - val_loss: 0.3997 - val_accuracy:

Able to get a result of 0.82 which we were able to get using 10 epochs for a max len of 80 and 20,000 max features.

---
### Machine Learning Foundation (C) 2020 IBM Corporation